In [50]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

True

In [51]:
# #Setting up the LangSmith
# #For now, all runs will be stored in the "KGBot Testing - GPT4"
# #If you want to separate the traces to have a better control of specific traces.
# #Metadata as llm version and temperature can be obtained from traces.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = (
    f"KGBot Testing - Interpreter_agent"  # Please update the name here if you want to create a new project for separating the traces.
)
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
from langsmith import Client

client = Client()

# #Check if the client was initialized
print(client)

Client (API URL: https://api.smith.langchain.com)


In [52]:
# langchain imports for agent and prompt handling
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser 
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain import hub

# langgraph imports for prebuilt tool invocation
from langgraph.prebuilt import ToolInvocation
from langgraph.graph import StateGraph, END

# langchain_core imports for message handling and action schema
from langchain_core.messages import BaseMessage, HumanMessage, FunctionMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
from langchain.schema import AgentAction, AgentFinish, HumanMessage

# langchain output parser for OpenAI functions
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from codeinterpreterapi import CodeInterpreterSession, File
# typing imports for type hinting
from typing import Annotated, List, Tuple, Union,  Any, Dict, Optional, Sequence, TypedDict
import operator
import functools

# Standard library imports for JSON and regular expressions
import json
import re
from importlib import reload
import sparql  
reload(sparql)
# Custom imports for RDF graph manipulation and chemical, target, taxon, and SPARQL resolution
from RdfGraphCustom import RdfGraph
from smile_resolver import smiles_to_inchikey
from chemical_resolver import ChemicalResolver
from target_resolver import target_name_to_target_id
from taxon_resolver import TaxonResolver
from sparql import GraphSparqlQAChain

# langchain pydantic for base model definitions
from langchain.pydantic_v1 import BaseModel, Field

# langchain tools for base, structured tool definitions, and tool decorators
from langchain.tools import BaseTool, StructuredTool, tool

# Standard library import for object serialization
import pickle

import sys
import logging





## Creating the Dataset for testing

In [70]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from langchain.evaluation import EvaluatorType
from langsmith.schemas import Example, Run
from langchain.smith import run_on_dataset, RunEvalConfig
dataset_inputs = [
    "How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?",
    #"Which samples have features (positive ionization mode) annotated as aspidosperma-type alkaloids by CANOPUS with a probability score above 0.5, ordered by the decreasing count of features as aspidosperma-type alkaloids?",
   # "Among the structural annotations from Tabernaemontana coffeoides (Apocynaceae) seeds extract, which ones contain an aspidospermidine substructure?",
    #"Among the SIRIUS structural annotations from Tabernaemontana coffeoides (Apocynaceae) seeds extract, which ones are reported in the Tabernaemontana genus in WD?",
    #"Which compounds annotated in the active extract of Melochia umbellata have activity against T. cruzi reported (in ChEMBL), and in which taxon they are reported (in Wikidata)?",
    #"Filter the positive ionization mode features of Melochia umbellata annotated as [M+H]+ by SIRIUS to keep the ones for which a feature in negative ionization mode is detected with the same retention time (± 3 seconds) and a mass corresponding to the [M-H]- adduct (± 5 ppm)",
    #"For features from Melochia umbellata in positive mode mode with SIRIUS annotations, get the ones for which a feature in negative ionization mode with the same retention time (± 3 sec) has the same SIRIUS annotation (2D IK)."
]

example_outputs = [
    "The number of features (in both positive and negative ionization modes) that have the same SIRIUS/CSI:FingerID and ISDB annotation, as determined by comparing the InCHIKey2D of the annotations, is 33,255.",
    #"32 samples. The sample with the highest count of features annotated as aspidosperma alkaloids (74) is from Tabernaemontana coffeoides (Apocynaceae) seeds extract.",
    #"3 distinct stereochemically undefined structures (2D InChiKey) that contain an aspidospermidine substructure: COC(=O)C1=C2Nc3ccccc3C23CCN2CC4(CC5CC67CC(=O)OC6CCN6CCC8(c9cccc(OC)c9N(C4)C58O)C67)C4OCCC4(C1)C23, COC(=O)C1CC23CCC[N+]4(C2C5(C1(CC3)NC6=CC=CC=C65)CC4)[O-], COC(=O)C1=C2Nc3ccccc3C23CCN2C3C3(CCOC3C3CC4CC56CCOC5CCN5CCC7(c8cccc(OC)c8NC47C32)C56)C1.",
    #"18 distinct stereochemically undefined structures annotated by SIRIUS in Tabernaemontana coffeoides (Apocynaceae) seeds extract and reported in at least one Tabernaemontana sp.",
    #"14 distinct stereochemically undefined structures, all of them reported in Waltheria indica.",
    #"62 features from Melochia umbellata in positive ionization mode annotated as [M+H]+ by SIRIUS with their corresponding potential [M-H]"
    #"22 features in positive ionization mode for which a feature in negative ionization mode with the same retention time has the same annotation."
]



# Creating the datasets for testing

dataset_name = "Testing q1_6"

dataset = client.create_dataset(
    dataset_name,
    description="An example dataset of questions to run",
)

client.create_examples(
    inputs=[{"question": question} for question in dataset_inputs],
    outputs=[{"output": answer} for answer in example_outputs],
    dataset_id=dataset.id,
)



HTTPError: [Errno 409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets] {"detail":"Dataset with this name already exists."}

In [66]:
print(dataset_name)

Testing q1_6


## Defining evaluator

In [67]:
# Defining a custom evaluator that checks if the generated answer is uninformative

@run_evaluator
def check_not_idk(run: Run, example: Example):
    """Illustration of a custom evaluator."""
    agent_response = run.outputs["output"]
    if "don't know" in agent_response or "not sure" in agent_response:
        score = 0
    else:
        score = 1
    # You can access the dataset labels in example.outputs[key]
    # You can also access the model inputs in run.inputs[key]
    return EvaluationResult(
        key="not_uncertain",
        score=score,
    )
# Defining the proper evaluation

evaluation_config = RunEvalConfig(
    # Evaluators can either be an evaluator type (e.g., "qa", "criteria", "embedding_distance", etc.) or a configuration for that evaluator
    evaluators=[
        # Measures whether a QA response is "Correct", based on a reference answer
        # You can also select via the raw string "qa"
        EvaluatorType.QA,
        # Measure the embedding distance between the output and the reference answer
        # Equivalent to: EvalConfig.EmbeddingDistance(embeddings=OpenAIEmbeddings())
        EvaluatorType.EMBEDDING_DISTANCE,
        # Grade whether the output satisfies the stated criteria.
        # You can select a default one such as "helpfulness" or provide your own.
        RunEvalConfig.LabeledCriteria("helpfulness"),
        # The LabeledScoreString evaluator outputs a score on a scale from 1-10.
        # You can use default criteria or write our own rubric
        RunEvalConfig.LabeledScoreString(
            {
                "accuracy": """
Score 1: The answer is completely unrelated to the reference.
Score 3: The answer has minor relevance but does not align with the reference.
Score 5: The answer has moderate relevance but contains inaccuracies.
Score 7: The answer aligns with the reference but has minor errors or omissions.
Score 10: The answer is completely accurate and aligns perfectly with the reference."""
            },
            #normalize_by=10,
        ),
    ],
    # You can add custom StringEvaluator or RunEvaluator objects here as well, which will automatically be
    # applied to each prediction. Check out the docs for examples.
    custom_evaluators=[check_not_idk],
)

In [68]:
from agent_supervisor import create_run_agent, process_stream

app = create_run_agent()
def evaluate_result(_input):
    message = {
                "messages": [
                    HumanMessage(content=_input["question"])  # Assuming q2 is the content of the message
                ]
            }
    response = app.invoke(message)
    return {"output": response}

2024-03-26 11:27:04,819 - INFO - query 
    SELECT DISTINCT ?cls ?com ?label
        WHERE {
            ?cls a rdfs:Class . 
            OPTIONAL { ?cls rdfs:comment ?com }
            OPTIONAL { ?cls rdfs:label ?label }
        }
        GROUP BY ?cls ?com ?label
    
Adding classes to graph: 100%|██████████| 50/50 [00:09<00:00,  5.40it/s]
2024-03-26 11:27:14,165 - INFO - namespaces [('brick', 'https://brickschema.org/schema/Brick#'), ('csvw', 'http://www.w3.org/ns/csvw#'), ('dc', 'http://purl.org/dc/elements/1.1/'), ('dcat', 'http://www.w3.org/ns/dcat#'), ('dcmitype', 'http://purl.org/dc/dcmitype/'), ('dcterms', 'http://purl.org/dc/terms/'), ('dcam', 'http://purl.org/dc/dcam/'), ('doap', 'http://usefulinc.com/ns/doap#'), ('foaf', 'http://xmlns.com/foaf/0.1/'), ('geo', 'http://www.opengis.net/ont/geosparql#'), ('odrl', 'http://www.w3.org/ns/odrl/2/'), ('org', 'http://www.w3.org/ns/org#'), ('prof', 'http://www.w3.org/ns/dx/prof/'), ('prov', 'http://www.w3.org/ns/prov#'), ('qb', 'http:

The namespace prefixes are: [('brick', 'https://brickschema.org/schema/Brick#'), ('csvw', 'http://www.w3.org/ns/csvw#'), ('dc', 'http://purl.org/dc/elements/1.1/'), ('dcat', 'http://www.w3.org/ns/dcat#'), ('dcmitype', 'http://purl.org/dc/dcmitype/'), ('dcterms', 'http://purl.org/dc/terms/'), ('dcam', 'http://purl.org/dc/dcam/'), ('doap', 'http://usefulinc.com/ns/doap#'), ('foaf', 'http://xmlns.com/foaf/0.1/'), ('geo', 'http://www.opengis.net/ont/geosparql#'), ('odrl', 'http://www.w3.org/ns/odrl/2/'), ('org', 'http://www.w3.org/ns/org#'), ('prof', 'http://www.w3.org/ns/dx/prof/'), ('prov', 'http://www.w3.org/ns/prov#'), ('qb', 'http://purl.org/linked-data/cube#'), ('schema', 'https://schema.org/'), ('sh', 'http://www.w3.org/ns/shacl#'), ('skos', 'http://www.w3.org/2004/02/skos/core#'), ('sosa', 'http://www.w3.org/ns/sosa/'), ('ssn', 'http://www.w3.org/ns/ssn/'), ('time', 'http://www.w3.org/2006/time#'), ('vann', 'http://purl.org/vocab/vann/'), ('void', 'http://rdfs.org/ns/void#'), ('wgs

## Defining chain_factory and running the evaluation

In [69]:

from uuid import uuid4
unique_id = uuid4().hex[0:8]

chain_results = run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=evaluate_result,
    evaluation=evaluation_config,
    verbose=True,
    project_name=f"Testing the app-{unique_id}",
    client=client,
    # Project metadata communicates the experiment parameters,
    # Useful for reviewing the test results
    # project_metadata={
    #     "env": "testing-notebook",
    #     "model": "gpt-4",
    #     "prompt": "5d466cbc",
    # },
)

2024-03-26 11:27:17,934 - INFO - Wrapping function (_input) as RunnableLambda.


View the evaluation results for project 'Testing the app-7e7e7e38' at:
https://smith.langchain.com/o/2830b3a1-2b4b-42fc-bc61-e5012f496ae5/datasets/094616d1-21f9-4a2d-b8f5-3fc704cd60cc/compare?selectedSessions=4126db01-c42f-4531-9147-db70718c7f39

View all tests for Dataset Testing q1_6 at:
https://smith.langchain.com/o/2830b3a1-2b4b-42fc-bc61-e5012f496ae5/datasets/094616d1-21f9-4a2d-b8f5-3fc704cd60cc


2024-03-26 11:27:18,368 - INFO - Wrapping function (_input) as RunnableLambda.


[>                                                 ] 0/1

2024-03-26 11:27:19,911 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-26 11:27:21,250 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2024-03-26 11:27:32,718 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated SPARQL:
PREFIX ns1: <https://enpkg.commons-lab.org/kg/>
PREFIX ns2: <https://enpkg.commons-lab.org/module/>
SELECT (COUNT(DISTINCT ?feature) AS ?count)
WHERE {
    ?feature a ns1:LCMSFeature .
    ?feature ns1:has_sirius_annotation ?siriusAnnotation .
    ?siriusAnnotation a ns1:SiriusStructureAnnotation .
    ?siriusAnnotation ns1:has_InChIkey2D ?inChIKey2D_sirius .
    
    ?feature ns1:has_isdb_annotation ?isdbAnnotation .
    ?isdbAnnotation a ns1:IsdbAnnotation .
    ?isdbAnnotation ns1:has_InChIkey2D ?inChIKey2D_isdb .
    
    FILTER (?inChIKey2D_sirius = ?inChIKey2D_isdb)
}
Saving results to file:  /var/folders/20/4kgcw5656h12ss_nj18mndwm0000gn/T/tmpono_otf8.csv

> Finished chain.


2024-03-26 11:27:35,308 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-26 11:27:59,856 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-26 11:27:59,885 - WARNING - Chain failed for example d1b9beb8-2267-4a31-945c-fa41a5ce12b2 with inputs {'question': 'How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?'}
Error Type: TypeError, Message: 'HumanMessage' object is not subscriptable


[------------------------------------------------->] 1/1

,error,execution_time,run_id
count,1,1.000000,1
unique,1,NaN,1
top,'HumanMessage' object is not subscriptable,NaN,984401a6-1010-42c7-9897-9fc989daf868
freq,1,NaN,1
mean,NaN,41.456267,NaN
std,NaN,NaN,NaN
min,NaN,41.456267,NaN
25%,NaN,41.456267,NaN
50%,NaN,41.456267,NaN
75%,NaN,41.456267,NaN


In [64]:
# Let's create the dictionary as described and extract the content

raw =  {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?"
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The number of features (in both positive and negative ionization modes) that have the same SIRIUS/CSI:FingerID and ISDB annotation, as determined by comparing the InCHIKey2D of the annotations, is 33,255.\n\n{\n  \"question\": \"How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?\",\n  \"generated_sparql_query\": \"PREFIX ns1: <https://enpkg.commons-lab.org/kg/>\\nSELECT (COUNT(DISTINCT ?feature) AS ?count)\\nWHERE {\\n    ?feature a ns1:LCMSFeature .\\n    ?feature ns1:has_sirius_annotation ?siriusAnnotation .\\n    ?feature ns1:has_isdb_annotation ?isdbAnnotation .\\n    ?siriusAnnotation ns1:has_InChIkey2D ?inChIKey2D1 .\\n    ?isdbAnnotation ns1:has_InChIkey2D ?inChIKey2D2 .\\n    FILTER (?inChIKey2D1 = ?inChIKey2D2)\\n}\",\n  \"file_path\": \"/var/folders/20/4kgcw5656h12ss_nj18mndwm0000gn/T/tmppow140s5.csv\"\n}",
          "name": "Sparql_query_runner"
        }
      }
    ],
    "next": "FINISH"
  }


# Then, extract and print the content from the specified location in the dictionary
extracted_content = raw["messages"][1]["kwargs"]["content"]
print(extracted_content)


The number of features (in both positive and negative ionization modes) that have the same SIRIUS/CSI:FingerID and ISDB annotation, as determined by comparing the InCHIKey2D of the annotations, is 33,255.

{
  "question": "How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?",
  "generated_sparql_query": "PREFIX ns1: <https://enpkg.commons-lab.org/kg/>\nSELECT (COUNT(DISTINCT ?feature) AS ?count)\nWHERE {\n    ?feature a ns1:LCMSFeature .\n    ?feature ns1:has_sirius_annotation ?siriusAnnotation .\n    ?feature ns1:has_isdb_annotation ?isdbAnnotation .\n    ?siriusAnnotation ns1:has_InChIkey2D ?inChIKey2D1 .\n    ?isdbAnnotation ns1:has_InChIkey2D ?inChIKey2D2 .\n    FILTER (?inChIKey2D1 = ?inChIKey2D2)\n}",
  "file_path": "/var/folders/20/4kgcw5656h12ss_nj18mndwm0000gn/T/tmppow140s5.csv"
}


In [ ]:
q1 = "How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey of the annotations?"
q1_bis = "How many features (pos ionization and neg ionization modes) have the same SIRIUS/CSI:FingerID and ISDB annotation by comparing the InCHIKey2D of the annotations?"
q2 = "Which extracts have features (pos ionization mode) annotated as the class, aspidosperma-type alkaloids, by CANOPUS with a probability score above 0.5, ordered by the decreasing count of features as aspidosperma-type alkaloids? Group by extract."
q3 = "Among the structural annotations from the Tabernaemontana coffeoides (Apocynaceae) seeds extract taxon , which ones contain an aspidospermidine substructure?"
q4 = "Among the SIRIUS structural annotations from the Tabernaemontana coffeoides (Apocynaceae) seeds extract, which ones are reported in the Tabernaemontana genus in Wikidata?"
q5 = "Which compounds have annotations with chembl assay results indicating reported activity against T. cruzi by looking at the cosmic, zodiac and taxo scores?"
q5_bis = "Which compounds have annotations with chembl assay results indicating reported activity against Trypanosoma cruzi by looking at the cosmic, zodiac and taxo scores?"
q6 = "Filter the pos ionization mode features of the Melochia umbellata taxon annotated as [M+H]+ by SIRIUS to keep the ones for which a feature in neg ionization mode is detected with the same retention time (+/- 3 seconds) and a mass corresponding to the [M-H]- adduct (+/- 5ppm)."
q7 = "For features from the Melochia umbellata taxon in pos ionization mode with SIRIUS annotations, get the ones for which a feature in neg ionization mode with the same retention time (+/- 3 seconds) has the same SIRIUS annotation by comparing the InCHIKey 2D. Return the features, retention times, and InChIKey2D"
q8 = "Which features were annotated as 'Tetraketide meroterpenoids' by SIRIUS, and how many such features were found for each species and plant part?"
q9 = "What are all distinct submitted taxons for the extracts in the knowledge graph?"
q10 = "What are the taxons, lab process and label (if one exists) for each sample? Sort by sample and then lab process"
q11 = "Count all the species per family in the collection"

q12 = "Taxons can be found in enpkg:LabExtract. Find the best URI of the Taxon in the context of this question : \n Among the structural annotations from the Tabernaemontana coffeoides (Apocynaceae) seeds extract taxon , which ones contain an aspidospermidine substructure, CCC12CCCN3C1C4(CC3)C(CC2)NC5=CC=CC=C45?"
q13 = "Which compounds annotated in the active extract of Melochia umbellata have activity against Trypanosoma cruzi reported (in ChEMBL)?"
q14 = "What are the variations in the concentration of key active compounds found in Tabernaemontana coffeoides seed extracts across different sample collections?"
q15 = "Which compounds are detected most in Tabernaemontana genus?"
q16 = (
    "What are the most frequently detected compounds in the leaves of the Tabernaemontana genus? "
    " over of features annotated as certain chemical classes vary across different Tabernaemontana genus extracts in the ENPKG, with a focus on features identified in positive ionization mode and annotated by CANOPUS with a probability score above 0.5?"
)
q17 = " For all the plant extracts plot the distribution of number of features per sample retention time vs mass to charge ratio"
q18 = "What are the most frequently observed chemical compounds in Tabernaemontana genus? Provide a bar chart."

In [ ]:
for s in app.stream(
    {"messages": [HumanMessage(content=q7)]},
    {"recursion_limit": 100},
):
    if "__end__" in s:
        print(s)
        print("----")

In [ ]:
for s in app.stream(
    {"messages": [HumanMessage(content=q2)]},
    {"recursion_limit": 100},
):
    if "__end__" in s:
        print(s)
        print("----")

In [ ]:
for s in app.stream(
    {"messages": [HumanMessage(content=q2)]},
    {"recursion_limit": 100},
):
    if "__end__" in s:
        print(s)
        print("----")

In [ ]:
for s in app.stream(
    {"messages": [HumanMessage(content=q6)]},
    {"recursion_limit": 100},
):
    if "__end__" in s:
        print(s)
        print("----")

In [ ]:
for s in app.stream(
    {"messages": [HumanMessage(content=q3)]},
    {"recursion_limit": 100},
):
    if "__end__" in s:
        print(s)
        print("----")

In [ ]:
def process_stream(app, q2):
    results = []  # Initialize an empty list to store results
    try:
        # Iterate over the stream from app.stream()
        for s in app.stream(
            {
                "messages": [
                    HumanMessage(
                        content=q2
                    )  # Assuming q2 is the content of the message
                ]
            },
            {"recursion_limit": 100},  # Additional options for the stream
        ):
            # Check if "__end__" is not in the stream output
            if "__end__" not in s:
                results.append(
                    s
                )  # Append the stream output to results list instead of printing
    except Exception as e:
        print(f"An error occurred: {e}")
    return results  # Return the list of results

In [ ]:
def process_stream(question):
    """
    Iterates over messages from app.stream(), printing each message until an "__end__" flag is encountered.

    :param question: The question to be sent as part of the initial stream request.
    """
    # Define the parameters for app.stream() as described
    stream_params = {
        "messages": [{"content": question}],
    }
    stream_options = {"recursion_limit": 100}

    try:
        for s in app.stream(stream_params, stream_options):
            if "__end__" not in s:
                print(s)
                print("----")
            else:
                break  # Exit the loop if "__end__" is encountered
    except Exception as e:
        print(f"An error occurred during streaming: {e}")